In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [53]:
BASE_URL = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok='

In [54]:
r = requests.get(BASE_URL+str(1)+'&page='+str(1))

In [55]:
soup = BeautifulSoup(r.text, 'lxml')

In [56]:
total_page_num = soup.select_one('td.pgRR > a')
total_page_num  = int(total_page_num.get('href').split('=')[-1])
total_page_num

30

In [57]:
ipt_html = soup.select_one('div.subcnt_sise_item_top')
global fields
fields = [item.get('value') for item in ipt_html.select('input') if item.get('value') in ['per', 'pbr', 'roe', 'market_sum', 'property_total', 'debt_total', 'sales', 'reserve_ratio']]
fields

['market_sum',
 'per',
 'property_total',
 'roe',
 'debt_total',
 'sales',
 'pbr',
 'reserve_ratio']

In [58]:
global fields
data = {
    'menu': 'market_sum',
    'returnUrl': BASE_URL + str(1) + "&page=" + str(1),
    'fieldIds': fields
}

r2 = requests.post('https://finance.naver.com/sise/field_submit.nhn', data=data)
r2.encoding='euc-kr'
r2.text

bl2.gif" width="15" height="13" alt="토론실"></a></td>\n\t\t\t\t</tr>\n\t\t\t\n\t\n\t\t\n\t\t\n\t\n\n\n\t\t\t\t<tr  onMouseOver="mouseOver(this)" onMouseOut="mouseOut(this)">\n\t\t\t\t\t<td class="no">43</td>\n\t\t\t\t\t<td><a href="/item/main.nhn?code=035900" class="tltle">JYP Ent.</a></td>\n\t\t\t\t\t<td class="number">34,900</td>\n\t\t\t\t\t<td class="number">\n\t\t\t\t<img src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" width="7" height="6" style="margin-right:4px;" alt="상승"><span class="tah p11 red02">\n\t\t\t\t200\n\t\t\t\t</span>\n\t\t\t</td>\n\t\t\t\t\t<td class="number">\n\t\t\t\t<span class="tah p11 red01">\n\t\t\t\t+0.58%\n\t\t\t\t</span>\n\t\t\t</td>\n\t\t\t\t\t<td class="number">500</td>\n\t\n\t\t\n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\t\t\t\t\t<td class="number">12,389</td>\n\t\t\t\n\t\t\n\t\n\t\t\n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\t\t\t\t\t<td class="number">1,444</td>\n\t\t\t\n\t\t\n\t\n\t\t\n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\t\t\t\t\t<td class="number">2,239</td

In [110]:
soup2 = BeautifulSoup(r2.text, 'lxml')
# table
table_html = soup2.select_one('table.type_2')
soup2
with open('test.txt', 'w') as f:
    f.write(str(soup2))

In [261]:
# column
header_data = [item.get_text().strip() for item in table_html.find('thead').find('tr').find_all('th') if item.get_text().strip() not in ['전일비', '등락률']]
header_data

['N',
 '종목명',
 '현재가',
 '액면가',
 '시가총액',
 '매출액',
 '자산총계',
 '부채총계',
 'PER',
 'ROE',
 'PBR',
 '유보율',
 '토론실']

In [262]:
del header_data[-1]

In [263]:
# 첫 줄은 필요없는 줄
td_items = [tr_item.find_all('td') for tr_item in table_html.find('tbody').find_all('tr')][1:]

In [342]:
inner_data = []
for item in td_items:
    for i in item[0:len(item)-1]:
        if i.find('a') or len(i.get('class', [])) > 0:
            child = i.findChildren()
            if len(child) >= 1:
                if child[0].get('href'):
                    inner_data.append(child[0].get_text().strip())
            else:
                inner_data.append(i.get_text().strip().replace(',', ''))

In [252]:
no_data = [i.get_text().strip() for item in td_items for i in item if len(i.get('class', [])) > 0 and ('no' in i.get('class', []))] # 번호

In [253]:
number_data = np.array(inner_data)

In [254]:
number_data.resize(len(no_data), len(header_data))

In [255]:
df = pd.DataFrame(data=number_data, columns=header_data)

In [379]:
pd_result = pd.read_excel('result.xlsx')

In [383]:
pd_result = pd_result.dropna()

In [384]:
pd_result.describe()

,N,현재가,액면가,상장주식수,시가총액,매출액,자산총계,부채총계,PER,ROE,PBR,유보율,BPS
count,775.000000,7.750000e+02,775.000000,7.750000e+02,7.750000e+02,7.750000e+02,7.750000e+02,7.750000e+02,775.000000,775.000000,775.000000,775.000000,775.000000
mean,480.409032,4.750273e+04,2089.290323,6.994901e+04,2.591846e+04,3.227523e+04,9.432642e+04,7.076774e+04,43.187342,-2.941329,1.715252,3167.330581,0.057786
std,296.477658,1.089494e+05,2074.784354,2.390911e+05,1.932658e+05,1.198899e+05,4.713243e+05,4.092072e+05,823.692149,53.104103,4.332177,8739.029755,0.166615
min,1.000000,2.580000e+02,0.000000,4.080000e+02,1.940000e+02,2.200000e+01,2.650000e+02,0.000000e+00,-6170.000000,-1165.400000,-4.710000,-1857.500000,-0.000589
25%,225.500000,4.430000e+03,500.000000,1.323850e+04,1.280500e+03,1.714000e+03,2.529000e+03,8.730000e+02,-2.565000,-1.780000,0.580000,282.100000,0.004302
50%,469.000000,1.220000e+04,500.000000,2.870500e+04,2.947000e+03,4.786000e+03,6.495000e+03,2.776000e+03,10.150000,3.840000,0.940000,1052.000000,0.013215
75%,723.500000,4.292500e+04,5000.000000,6.299350e+04,9.454000e+03,1.659050e+04,2.355250e+04,1.180800e+04,22.800000,8.935000,1.865000,2684.800000,0.049768
max,1177.000000,1.570000e+06,5000.000000,5.969783e+06,5.062376e+06,2.368070e+06,6.106722e+06,5.673107e+06,21361.100000,61.960000,107.420000,127125.600000,2.996496


In [389]:
pd_result['BPS'] = ((pd_result['자산총계'] - pd_result['부채총계']) / (pd_result['상장주식수']) * 100)

<ipython-input-389-0c8ccd3e77b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_result['BPS'] = ((pd_result['자산총계'] - pd_result['부채총계']) / (pd_result['상장주식수']) * 100)


In [398]:
pd_result['부채비율'] = (pd_result['부채총계']/pd_result['자산총계']) * 100

In [399]:
pd_result['부채비율'].head()

0    27.043375
1    27.067225
2    51.481116
4    54.600388
5    63.533262
Name: 부채비율, dtype: float64

In [401]:
pd_result['BPS'].head()

0     46.224126
1     71.303513
2     50.255383
4    266.183137
5    357.287942
Name: BPS, dtype: float64

In [400]:
pd_result[(pd_result['PER'] < 10.0) & (pd_result['PBR'] < 1.0) & (pd_result['ROE'] > 5.0) & (pd_result['시가총액'] > 1.0 ) & (pd_result['부채비율'] < 100.0)]

,N,종목명,현재가,액면가,상장주식수,시가총액,매출액,자산총계,부채총계,PER,ROE,PBR,유보율,BPS,부채비율
17,18,KB금융,53900,5000,415808,224120,556802.0,6106722.0,5673107.0,6.49,8.53,0.49,2028.5,104.282505,92.899382
21,22,신한지주,36350,5000,516600,187784,290007.0,6052341.0,5588772.0,5.39,8.20,0.44,1483.4,89.734611,92.340666
28,29,하나금융지주,41150,5000,300242,123550,482159.0,4603133.0,4287130.0,4.68,8.96,0.39,2025.0,105.249432,93.135045
46,47,우리금융지주,10000,5000,722268,72227,286376.0,3990810.0,3723552.0,5.52,5.87,0.31,640.0,37.002608,93.303164
49,50,기업은행,9040,5000,738664,66775,185746.0,3616162.0,3364738.0,4.51,6.44,0.27,501.0,34.037668,93.047214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,909,미래아이앤지,530,100,129484,686,58.0,923.0,29.0,3.49,24.75,0.76,597.9,0.690433,3.141928
927,928,MH에탄올,8660,500,7379,639,946.0,4497.0,3693.0,9.36,11.49,0.92,1500.7,10.895785,82.121414
973,974,엔케이물산,680,100,79439,540,341.0,690.0,69.0,7.91,7.04,0.87,700.1,0.781732,10.000000
975,976,원림,24500,5000,2200,539,684.0,1773.0,439.0,3.52,16.31,0.48,913.9,60.636364,24.760293
